In [ ]:
# This notebook writes data to the Neo4j graph.

In [1]:
from neo4j import GraphDatabase

In [2]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"

In [3]:
driver = GraphDatabase.driver(uri, auth=(user, password))

In [4]:
session = driver.session()

In [ ]:
# loading phone nodes into graph; csv needs to be in graph's import folder
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///phone_nodes.csv' AS row
    WITH row
    CALL {
        WITH row
        CREATE (:Identifier {
            identifier: row.identifier,
            type: row.type
            })
    };''')

In [5]:
# create an index for phone identifiers
session.run(
    '''CREATE INDEX identifier_range_index FOR (i:Identifier) ON (i.identifier);''')

In [6]:
# loading phone relationships into graph
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///phone_relationships_v1.csv' AS row
    WITH row
    CALL {
        WITH row
        MATCH (a:Identifier), (b:Identifier) WHERE a.identifier = row.source and b.identifier = row.target
        MERGE (a)-[:CONTACT {first_seen:row.first_seen, last_seen:row.last_seen, times:row.times, collection:row.collection}]->(b)
    };''')